# Домашняя работа
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек 

2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip3 install xlrd

In [3]:
df = pd.read_excel('отзывы за лето.xls')

In [4]:
df.head(15)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [5]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [6]:
df.shape

(20659, 3)

In [7]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import pymorphy2
import re

In [23]:
morph = pymorphy2.MorphAnalyzer()
stopwords_list = stopwords.words('russian')
punct = set(punctuation)

def tokenize_and_preprocess(text):
    if type(text) is not str:
        return []
    # txt = "".join(c for c in text if c not in punct)
    txt = text
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    words = word_tokenize(txt)
    lemmas = [morph.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords_list and w.isalpha()]

tokenize_and_preprocess('Съешь еще этих мягких французских булок, да выпей же чаю!')

['съесть', 'ещё', 'мягкий', 'французский', 'булка', 'выпить', 'чай']

In [24]:
df['Prepared_Content'] = df['Content'].apply(tokenize_and_preprocess)
df.head(5)

,Rating,Content,Date,Prepared_Content
0,5,It just works!,2017-08-14,"[it, just, works]"
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,"[целое, удобноной, приложение, минус, хотеть, ..."
2,5,Отлично все,2017-08-14,"[отлично, всё]"
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,"[стать, зависать, работа, антивирус, далёкий, ..."
4,5,"Очень удобно, работает быстро.",2017-08-14,"[очень, удобно, работать, быстро]"


In [25]:
from sklearn.model_selection import train_test_split

In [26]:
df_train, df_val = train_test_split(df, test_size=0.25, shuffle=True)

In [35]:
corpus_tokens = []
for i in range(len(df_train)):
    corpus_tokens += df_train["Prepared_Content"].iloc[i]

In [36]:
num_classes = 5
max_words = 5000
max_len = 40

In [37]:
from nltk.probability import FreqDist
dist = FreqDist(corpus_tokens)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [38]:
tokens_filtered_top[:10]

['приложение',
 'всё',
 'очень',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'спасибо',
 'хороший',
 'нравиться']

In [39]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [45]:
def text_to_sequence(tokens, maxlen):
    result = []
    for word in tokens:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0] * (maxlen-len(result))
    return padding + result[-maxlen:]

In [46]:
X_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Prepared_Content"]], dtype=np.int32)
X_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Prepared_Content"]], dtype=np.int32)

In [58]:
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras

In [56]:
label_encoder = preprocessing.LabelEncoder()
y_train = label_encoder.fit_transform(df_train['Rating'])
y_val = label_encoder.fit_transform(df_val['Rating'])

In [59]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

In [32]:
epochs = 20
batch_size = 512
print_batch_n = 100

In [62]:
model = keras.models.Sequential([
    keras.layers.Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    keras.layers.Conv1D(128, 10),
    keras.layers.Activation("relu"),
    keras.layers.GlobalMaxPool1D(),
    keras.layers.Dense(25),
    keras.layers.Activation("relu"),
    keras.layers.Dense(num_classes),
    keras.layers.Activation('softmax'),
])

2022-08-28 23:22:49.287347: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-28 23:22:49.291369: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [63]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [68]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_val, y_val))

Epoch 1/20


2022-08-28 23:25:58.711263: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-28 23:26:00.920649: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - 7s 114ms/step - loss: 1.0529 - accuracy: 0.6945 - val_loss: 0.8683 - val_accuracy: 0.7082
Epoch 2/20


2022-08-28 23:26:05.479907: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - 3s 103ms/step - loss: 0.7515 - accuracy: 0.7538 - val_loss: 0.6782 - val_accuracy: 0.7652
Epoch 3/20
31/31 [==============================] - 3s 105ms/step - loss: 0.6245 - accuracy: 0.7805 - val_loss: 0.6384 - val_accuracy: 0.7766
Epoch 4/20
31/31 [==============================] - 3s 101ms/step - loss: 0.5703 - accuracy: 0.7995 - val_loss: 0.6375 - val_accuracy: 0.7793
Epoch 5/20
31/31 [==============================] - 3s 105ms/step - loss: 0.5226 - accuracy: 0.8175 - val_loss: 0.6211 - val_accuracy: 0.7839
Epoch 6/20
31/31 [==============================] - 3s 99ms/step - loss: 0.4766 - accuracy: 0.8370 - val_loss: 0.6287 - val_accuracy: 0.7839
Epoch 7/20
31/31 [==============================] - 3s 100ms/step - loss: 0.4314 - accuracy: 0.8568 - val_loss: 0.6456 - val_accuracy: 0.7845
Epoch 8/20
31/31 [==============================] - 3s 98ms/step - loss: 0.3838 - accuracy: 0.8787 - val_loss: 0.6752 - val_accuracy: 0.7729
Epoch 9/20
31/31 [=

### Готовые веса

https://keras.io/examples/nlp/pretrained_word_embeddings/

In [77]:
embeddings_index = {}
with open('180/model.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        word = word.split('_')[0]
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 154617 word vectors.


In [78]:
num_tokens = len(vocabulary) + 2
embedding_dim = 300
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in vocabulary.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 3304 words (1695 misses)


In [85]:
model_pt = keras.models.Sequential([
    keras.layers.Embedding(
        input_length=max_len,
        input_dim=max_words + 1,
        output_dim=embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        trainable=False,
    ),
    keras.layers.Conv1D(128, 10),
    keras.layers.Activation("relu"),
    keras.layers.GlobalMaxPool1D(),
    keras.layers.Dense(25),
    keras.layers.Activation("relu"),
    keras.layers.Dense(num_classes),
    keras.layers.Activation('softmax'),
])

In [86]:
model_pt.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [87]:
history = model_pt.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose=1,
                    validation_data=(X_val, y_val))

Epoch 1/20


2022-08-29 13:10:00.933225: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - ETA: 0s - loss: 0.9744 - accuracy: 0.7157

2022-08-29 13:10:04.007957: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - 4s 65ms/step - loss: 0.9744 - accuracy: 0.7157 - val_loss: 0.8129 - val_accuracy: 0.7344
Epoch 2/20
31/31 [==============================] - 1s 44ms/step - loss: 0.6827 - accuracy: 0.7822 - val_loss: 0.7532 - val_accuracy: 0.7454
Epoch 3/20
31/31 [==============================] - 1s 44ms/step - loss: 0.5730 - accuracy: 0.8192 - val_loss: 0.7411 - val_accuracy: 0.7508
Epoch 4/20
31/31 [==============================] - 1s 45ms/step - loss: 0.4862 - accuracy: 0.8537 - val_loss: 0.7527 - val_accuracy: 0.7535
Epoch 5/20
31/31 [==============================] - 1s 45ms/step - loss: 0.4223 - accuracy: 0.8746 - val_loss: 0.7648 - val_accuracy: 0.7483
Epoch 6/20
31/31 [==============================] - 1s 44ms/step - loss: 0.3694 - accuracy: 0.8925 - val_loss: 0.7954 - val_accuracy: 0.7431
Epoch 7/20
31/31 [==============================] - 1s 44ms/step - loss: 0.3307 - accuracy: 0.9041 - val_loss: 0.8355 - val_accuracy: 0.7491
Epoch 8/20
31/31 [======

__Вывод:__ обычная модель и с преобученными весами эмбединга одинаково хорошо проявили себя на валидационной выборке, обычная даже немного лучше.